# 0. Importing libraries, mergning raw files, exporting merged "master file" and reading it in for cleaning


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import pyarrow.parquet as pq
import os
import glob
from pathlib import Path

In [ ]:
# glob = specific folder
# rglob = including subfolder

input_dir = Path(r'C:\Users\ insert your path')
files = list(input_dir.glob("*.csv*"))
len(files)

In [ ]:
# Store all dataframes in a list

parts = []
for path in list(input_dir.glob("*.csv*")):
    part = pd.read_csv(path)
    parts.append(part)

In [ ]:
masterfile = pd.concat(parts)

# Save merged dataframe to outpit directory

masterfile.to_csv(r'C:\Users\ insert your path \ DKMasterSale.csv', index=False)

In [ ]:
# importing masterfile as df

df = pd.read_csv(r'C:\Users\ insert your path \DKMasterSale.csv', low_memory=False)

In [ ]:
df.info()

# as seen memory usage is large. we'll work on that during the cleaning and finetuning

# 1. Investigating data

In [ ]:
# we'll have a look at the df to get a sense of the data

df.head(3)

In [ ]:
# checking shape

df.shape

In [ ]:
# checking duplicates (will revert on this later)

df[df.duplicated(['Boligtype & Adresse'], keep=False)]

In [ ]:
# checking dtypes()

df.dtypes

In [ ]:
# checking for na's

df.isnull().sum()

# as seen we have quite a few na's due to "double" columns

In [ ]:
# creating 4 new columns by combining "double" columns

df['purchase_price'] = df['KÃ¸besum'].combine_first(df['Købesum'])
df['sqm'] = df['mÂ² & Kr. / mÂ²'].combine_first(df['m² & Kr. / m²'])
df['no_rooms'] = df['VÃ¦r.'].combine_first(df['Vær.'])
df['year_build'] = df['ByggeÃ¥r'].combine_first(df['Byggeår'])

In [ ]:
# dropping 8 columns as we have created the new 4 columns. dropping "Unnamed: 7" as well as it does not have any numerical value we can use

df = df.drop(['KÃ¸besum', 'Købesum', 'mÂ² & Kr. / mÂ²', 'm² & Kr. / m²', 'VÃ¦r.', 'Vær.', 'ByggeÃ¥r', 'Byggeår', 'Unnamed: 7'], axis=1)

In [ ]:
# renaming columns (Danish to English)

df.rename(columns = {"Boligtype & Adresse" : "type_and_address" , 
                      'Dato & Type' : 'date_and_sales_type' ,                    
                      "Den procentuelle forskel mellem seneste udbudspris og salgsprisen %" : "%_change_between_offer_and_purchase"}, inplace=True)

In [ ]:
# checking df

df.head(3)

In [ ]:
# # checking for na's again. there's only nans in "%_change_between_offer_and_purchase". we'll get back to that

df.isnull().sum()

# 2. Split data and cleaning colums (date_and_sales_type column)

In [ ]:
# we'll start by creating a seperate date column

df["date"] = df["date_and_sales_type"].str[0:10]

In [ ]:
# convert dtype of new "date" column from object to datetime

df["date"] = pd.to_datetime(df["date"], dayfirst=False)

In [ ]:
# we'll create a seperate sales_type column

df["sales_type"] = df["date_and_sales_type"].str[10:]

In [ ]:
# checking unique string types in "sales_type"

df["sales_type"].unique()

In [ ]:
# converting unique string types in "sales_type" from Danish to English

df["sales_type"] = df["sales_type"].str.replace("Alm. Salg" , "regular_sale")
df["sales_type"] = df["sales_type"].str.replace("Fam. Salg" , "family_sale")
df["sales_type"] = df["sales_type"].str.replace("Andet" , "other_sale")
df["sales_type"] = df["sales_type"].str.replace("Auktion" , "auction")

In [ ]:
# check convert unique string types in "sales_type"

df["sales_type"].unique()

In [ ]:
# changing dtype of "sales_type" column from object to category to reduce memory usage

df["sales_type"] = df["sales_type"].astype("category")

In [ ]:
# show number of different sale_types

df["sales_type"].value_counts()

In [ ]:
# dropping "date_and_sales_type" column as we have created the 2 new columns

df = df.drop("date_and_sales_type", axis=1)

# we are done with the orginal "date_and_sales_type" and we now have two seperate columns

# 3. Split data and cleaning colums (purchase_price column)

In [ ]:
# deleting ".kr" and "." in column purchase_price as we want to convert the column to a numeric column

df["purchase_price"] = df["purchase_price"].str.replace("kr." , "")
df["purchase_price"] = df["purchase_price"].str.replace("." , "")
df["purchase_price"] = df["purchase_price"].str.strip()   # strips white space

In [ ]:
# converting to numeric and setting int to int32 to save memory

df["purchase_price"] = pd.to_numeric(df["purchase_price"].astype(np.int32))

# we are done with the "purchase_price" column

In [ ]:
df["purchase_price"]

In [ ]:
# checking for outlines in purchase_price

df["purchase_price"].plot(kind="box")

# as seen we have a some extreme purchase_prices

In [ ]:
# checking purchase_price greater than DKK 50,000,000 as these most likely are incorrect

df[df["purchase_price"] > 50_000_000]

In [ ]:
# dropping rows where purchase_price is greater than 50,000,000

df = df.drop(df[df["purchase_price"] > 50_000_000].index)

In [ ]:
# checking purchase_price less than DKK 250,000 as these most likely are incorrect

df[df["purchase_price"] < 250_000]

In [ ]:
# dropping rows where purchase_price is less than DKK 250,000

df = df.drop(df[df["purchase_price"] <= 250_000].index)

In [ ]:
# checking na's

df["purchase_price"].isnull().sum()

In [ ]:
# checking distribution of purchase_price using defined bins

df["purchase_price"].value_counts(bins=[0, 250000, 500000, 1000000, 1500000, 2000000, 2500000, 3000000, 3500000, 4000000, 4500000, 5000000, 7500000, 10000000, 20000000, 50000000, 75000000], sort=False)

# and we are done with this column

In [ ]:
# checking df

df.head(3)

# 4. Split data and cleaning colums (sqm column)

In [ ]:
# select string from left to right and deleting unwanted text

df["sqm"] = df["sqm"].str[0:7]
df["sqm"] = df["sqm"].str.replace(" mÂ²" , "")
df["sqm"] = df["sqm"].str.replace("m²" , "")
df["sqm"] = df["sqm"].str.replace("mÂ" , "")
df["sqm"] = df["sqm"].str.replace("0k" , "")
df["sqm"] = df["sqm"].str.replace(" " , "") # removes white inside
df["sqm"] = df["sqm"].str.strip()   # strips white space outside 

In [ ]:
# convert to numeric column and we are done

df["sqm"] = pd.to_numeric(df["sqm"])

In [ ]:
# convert from float64 to float32 to save memory

df["sqm"] = df["sqm"].astype(np.float32)

In [ ]:
df["sqm"]

In [ ]:
# checking sqm

df["sqm"].plot(kind="box")

# as seen we have some outlines

In [ ]:
# chekcing sqm > 1000 as these are most likely incorrect

df[df["sqm"] > 1000]

In [ ]:
# dropping rows with sqm greater than 1000

df = df.drop(df[df["sqm"] > 1000].index)

In [ ]:
# chekcing sqm less than or equal to 25 as these are most likely incorrect

df[df["sqm"] <= 25]

In [ ]:
# dropping rows with sqm less than or equal to 25

df = df.drop(df[df["sqm"] <= 25].index)

In [ ]:
# chekcng df

df.head(3)

# 5. Split data and cleaning colums (%_change_between_offer_and_purchase column)

In [ ]:
# deleting unwanted text in _change_between_offer_and_purchase column

df["%_change_between_offer_and_purchase"] = df["%_change_between_offer_and_purchase"].str.replace("%" , "")
df["%_change_between_offer_and_purchase"] = df["%_change_between_offer_and_purchase"].str.replace("." , "")

In [ ]:
# convert to numeric column

df["%_change_between_offer_and_purchase"] = pd.to_numeric(df["%_change_between_offer_and_purchase"])

In [ ]:
# convert from float64 to float32 to save memory

df["%_change_between_offer_and_purchase"] = df["%_change_between_offer_and_purchase"].astype(np.float32)

In [ ]:
# checking value range in %_change_between_offer_and_purchase (ckecking for outlines)

df["%_change_between_offer_and_purchase"].value_counts(sort=True)

In [ ]:
# plotting value range in %_change_between_offer_and_purchase (ckecking for outlines)

df["%_change_between_offer_and_purchase"].plot(kind="box")

In [ ]:
# checking number of rows with change greater than 49%

df["%_change_between_offer_and_purchase"].loc[df["%_change_between_offer_and_purchase"] > 49]

In [ ]:
# dropping rows with change ahove 49% (https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression)

df = df.drop(df[df["%_change_between_offer_and_purchase"] > 49].index)

In [ ]:
# checking number of rows with change greater than -49%

df["%_change_between_offer_and_purchase"].loc[df["%_change_between_offer_and_purchase"] < -49]

In [ ]:
# dropping rows with change greater than -49%

df = df.drop(df[df["%_change_between_offer_and_purchase"] < -49].index)

In [ ]:
df.shape

In [ ]:
# checking "buckets" of change

df["%_change_between_offer_and_purchase"].value_counts(bins=20)

In [ ]:
# visual checking "buckets" of change

df["%_change_between_offer_and_purchase"].plot(kind="hist")

In [ ]:
# checking for NaNs in %_change_between_offer_and_purchase

df["%_change_between_offer_and_purchase"].isna().sum()

In [ ]:
# replaceing NaN with 0 as there was no difference better offer price and purchase price

df['%_change_between_offer_and_purchase'] = df['%_change_between_offer_and_purchase'].replace(np.nan, 0)


In [ ]:
# checking buckets

df["%_change_between_offer_and_purchase"].value_counts(bins=20).sort_index()


In [ ]:
df.head(3)

# 6. Split data and cleaning colums (type_and_address column)

In [ ]:
# spliting text to get an overview and creating a variable (var)

var = df["type_and_address"].str.split(" ", n=11, expand=True)

In [ ]:
# checking var

var.head(3)

In [ ]:
# seeems like we have some double text

var[1].value_counts()

In [ ]:
# seeems like we have some double text (checking again)

df["type_and_address"]

In [ ]:
# deleting housing type "duplicates" in each string

df["type_and_address"] = df["type_and_address"].str.replace("VVilla " , "" )
df["type_and_address"] = df["type_and_address"].str.replace("EEjerlejlighed " , "" )
df["type_and_address"] = df["type_and_address"].str.replace("FFritidshus " , "" )
df["type_and_address"] = df["type_and_address"].str.replace("RRÃ¦kkehus " , "")
df["type_and_address"] = df["type_and_address"].str.replace("LLandejendom " , "" )
df["type_and_address"] = df["type_and_address"].str.replace("RRækkehus " , "" )

In [ ]:
# checking cleaned data

var = df["type_and_address"].str.split(" ", n=9, expand=True)

In [ ]:
# checking cleaned data

var[0].value_counts()

In [ ]:
# replacing DK text with ENG text

df["type_and_address"] = df["type_and_address"].str.replace("Villa " , "Villa " )
df["type_and_address"] = df["type_and_address"].str.replace("Ejerlejlighed " , "Apartment " )
df["type_and_address"] = df["type_and_address"].str.replace("Fritidshus " , "Summerhouse " )
df["type_and_address"] = df["type_and_address"].str.replace("RÃ¦kkehus " , "Townhouse ")
df["type_and_address"] = df["type_and_address"].str.replace("Landejendom " , "Farm " )
df["type_and_address"] = df["type_and_address"].str.replace("Rækkehus " , "Summerhouse " )

In [ ]:
var

In [ ]:
# we want split the column in order to create a new column called "house_type"

df["type_and_address"].str.split(" ", n=1, expand=True)

In [ ]:
# creating new column with house_type

new_col = df["type_and_address"].str.split(" ", n=1, expand=True)
df["house_type"] = new_col[0]

In [ ]:
# checking unique "strings"

df["house_type"].unique()

In [ ]:
# change from dtype object to category to save memory

df["house_type"] = df["house_type"].astype("category")

In [ ]:
# deleting house_type text from "type_and_address" column

df["type_and_address"] = df["type_and_address"].str.replace("Villa " , "" )
df["type_and_address"] = df["type_and_address"].str.replace("Apartment " , "" )
df["type_and_address"] = df["type_and_address"].str.replace("Summerhouse " , "" )
df["type_and_address"] = df["type_and_address"].str.replace("Townhouse " ,  "")
df["type_and_address"] = df["type_and_address"].str.replace("Farm " , "")

In [ ]:
# replacing odd charectars with DK letters

df["type_and_address"] = df["type_and_address"].str.replace("Ã¦" , "æ")
df["type_and_address"] = df["type_and_address"].str.replace("Ã¸" , "ø")
df["type_and_address"] = df["type_and_address"].str.replace("Ã¥" , "å")
df["type_and_address"] = df["type_and_address"].str.replace("Ã" , "A")
df["type_and_address"] = df["type_and_address"].str.replace("-Ã" , "A")

In [ ]:
# checking type_and_address split as we want to seperate on address, zip code and city

df["type_and_address"].str.split(" ", n=8, expand=True)

In [ ]:
# checking names list

names = df["type_and_address"].tolist()

In [ ]:
#names

In [ ]:
# we can split on double space thus...:

df["type_and_address"].str.split("  ", n=2, expand=True)

In [ ]:
# creating new 2 columns for adress and zip code and city

new_cols = new_col = df["type_and_address"].str.split("  ", n=2, expand=True)
df["address"] = new_cols[0]
df["city_test"] = new_cols[1]

In [ ]:
# creating new column for zip_code

df["zip_code"] = df["city_test"].str[0:4]

In [ ]:
# convert to numeric column

df["zip_code"] = pd.to_numeric(df["zip_code"])

In [ ]:
df["zip_code"].info()

In [ ]:
df["zip_code"] = df["zip_code"].astype(np.int16)

In [ ]:
# creating new column for city

df["city"] = df["city_test"].str[5:]

In [ ]:
# dropping columns we do not need any longer

df = df.drop(['type_and_address', 'city_test'], axis=1)

In [ ]:
# checking df

df.head(3)

# 7. Split data and cleaning colums (no_rooms column)

In [ ]:
# checking no_romms

df["no_rooms"].plot(kind="box")

# we have outliers and want to delete those

In [ ]:
# chekcing no_rooms > 15

df[df["no_rooms"] > 15]

In [ ]:
# dropping rows with no_rooms greater than 15

df = df.drop(df[df["no_rooms"] > 15].index)

In [ ]:
df[df["no_rooms"] < 1]

In [ ]:
# dropping rows with no_rooms less than 1

df = df.drop(df[df["no_rooms"] < 1].index)

In [ ]:
# change dtype

df["no_rooms"] = df["no_rooms"].astype("int8")

# 8. Split data and cleaning colums (year_build column)

In [ ]:
# checking for outlines in year build

df["year_build"].plot(kind="box")

In [ ]:
# chekcing year_build less than 1000

df[df["year_build"] < 1000]

In [ ]:
# dropping rows with year_build less than 1000

df = df.drop(df[df["year_build"] < 1000].index)

In [ ]:
# convert to int16

df["year_build"] = df["year_build"].astype("int16")

# 9. Further checks and finetuning

In [ ]:
# create a new column call sqm_price

df["sqm_price"] = df["purchase_price"] / df["sqm"]

In [ ]:
# converting dtype to save memory

df["sqm_price"] = df["sqm_price"].astype("float32")

In [ ]:
df["sqm_price"].plot(kind="box")

In [ ]:
# checking for outlines

df[df["sqm_price"] > 75_000]

In [ ]:
# dropping rows with sqm_price greater than than DKK 75000

df = df.drop(df[df["sqm_price"] > 75000].index)

In [ ]:
df.head(3)

In [ ]:
# checkng dtypes

df.dtypes

In [ ]:
# adding a quater column 

df["quarter"] = pd.PeriodIndex(df["date"], freq="Q")

In [ ]:
# adding an unique house_id

df["house_id"] = df.groupby(['address', 'zip_code'], sort = False).ngroup()

In [ ]:
# reordering columns

neworder = ['date', 'quarter', 'house_id', 'house_type', 'sales_type', 'year_build', 'purchase_price', '%_change_between_offer_and_purchase' , 'no_rooms', 'sqm', 'sqm_price', 'address', 'zip_code' , 'city']

df=df.reindex(columns=neworder)

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
# checking for duplicates with same sales date

df[df.duplicated(["address", "zip_code", "date" ], keep=False)]

In [ ]:
# duplicates with same sales date i.e. only one row

df = df.drop_duplicates(subset=['address', 'zip_code', 'date'], keep='first')

In [ ]:
df.shape

In [ ]:
df["house_id"] = df.groupby(['address', 'zip_code'], sort = False).ngroup()

In [ ]:
df

In [ ]:
df[df.duplicated('house_id', keep=False)]

In [ ]:
df.loc[df['house_id'] == 77891]

In [ ]:
###https://www.geeksforgeeks.org/delete-duplicates-in-a-pandas-dataframe-based-on-two-columns/

In [ ]:
df.sort_values(by=['house_id', 'date'])

In [ ]:
df['date_diff'] = df.groupby('house_id')['date'].diff().dt.days

In [ ]:
df['date_diff'].isna().sum()

In [ ]:
df_cleaned = df[df['date_diff'].isna() | (df['date_diff'] >= 14)]

In [ ]:
df = df_cleaned.drop(columns=['date_diff'])

In [ ]:
df

# 10. Importing zip code data file

In [ ]:
zd = pd.read_excel(r'C:\Users\ insert your path \DK_regions_zip_codes.xlsx')

In [ ]:
zd.dtypes

In [ ]:
# changing dtype to save memory

zd["area"] = zd["area"].astype("category")
zd["region"] = zd["region"].astype("category")

In [ ]:
df.shape

In [ ]:

# Define a function to assign the region_name and region_no based on zip_code

# OBS! SLOW code however couldn't get dirc. version to work

def assign_region(row):
    if 0 <= row['zip_code'] <= 999:
        return pd.Series(['Special for organisation and large corportes', 'Non given'], index=['area', 'region'])
    elif 1000 <= row['zip_code'] <= 2999:
        return pd.Series(['Capital, Copenhagen', 'Zealand'], index=['area', 'region'])
    elif 3000 <= row['zip_code'] <= 3699:
        return pd.Series(['North Zealand', 'Zealand'], index=['area', 'region'])
    elif 3700 <= row['zip_code'] <= 3799:
        return pd.Series(['Bornholm', 'Bornholm'], index=['area', 'region'])
    elif 3800 <= row['zip_code'] <= 3899:
        return pd.Series(['Faroe Islands', 'Faroe Islands'], index=['area', 'region'])
    elif 3900 <= row['zip_code'] <= 3999:
        return pd.Series(['Greenland', 'Greenland'], index=['area', 'region'])
    elif 4000 <= row['zip_code'] <= 4999:
        return pd.Series(['Other islands', 'Zealand'], index=['area', 'region'])
    elif 5000 <= row['zip_code'] <= 5999:
        return pd.Series(['Fyn & islands', 'Fyn & islands'], index=['area', 'region'])
    elif 6000 <= row['zip_code'] <= 6999:
        return pd.Series(['South jutland', 'Jutland'], index=['area', 'region'])
    elif 7000 <= row['zip_code'] <= 7999:
        return pd.Series(['South jutland', 'Jutland'], index=['area', 'region'])
    elif 8000 <= row['zip_code'] <= 8999:
        return pd.Series(['East & mid jutland', 'Jutland'], index=['area', 'region'])
    elif 9000 <= row['zip_code'] <= 9999:
        return pd.Series(['North jutland', 'Jutland'], index=['area', 'region'])
    else:
        return pd.Series([None, None], index=['region_name', 'region_no'])

# Apply the function to the DataFrame
df[['area', 'region']] = df.apply(assign_region, axis=1)

In [ ]:
df.shape

In [ ]:
df["region"].unique()

In [ ]:
df

# 11. Importing interest rate data file

In [ ]:
ird = pd.read_excel(r'C:\Users\ insert your path \DK_interest_rates.xlsx')

In [ ]:
# replacing in order to convert to data format

ird["date"] = ird["date"].str.replace("M" , "-" )
ird["date"] = ird["date"].str.replace("D" , "-" )

In [ ]:
# date convert

ird["date"] = pd.to_datetime(ird["date"], dayfirst=False)


In [ ]:
ird["quarter"] = pd.PeriodIndex(ird["date"], freq="Q")

In [ ]:
ird = ird.drop('date', axis=1)

In [ ]:
ird["quarter"].info()

In [ ]:
ird.dtypes

In [ ]:
ird

In [ ]:
#ird.sort_index(ascending=False)

In [ ]:
# Ensure there are no duplicates in 'ird' for the 'quarter' column
ird_unique = ird.drop_duplicates(subset='quarter')

# Now, perform the merge
df = df.merge(ird_unique[['quarter', 'nom_interest_rate%']], on='quarter', how='left')

In [ ]:
df

# 12. Importing inflation rate data file

In [ ]:
inflation = pd.read_excel(r'C:\Users\ insert your path \DK_inflation_rates.xlsx')

In [ ]:
inflation.head(3)

In [ ]:
inflation["dk_ann_infl_rate%"] = inflation["dk_ann_infl_rate%"].str.replace(" %" , "" )
inflation["dk_ann_infl_rate%"] = pd.to_numeric(inflation["dk_ann_infl_rate%"])

In [ ]:
inflation["quarter"] = pd.PeriodIndex(inflation["date"], freq="Q")

In [ ]:
inflation.head(3)

In [ ]:
inflation.drop("date", axis=1)

In [ ]:
# Ensure there are no duplicates in 'ird' for the 'quarter' column
inflation_unique = inflation.drop_duplicates(subset='quarter')

# Now, perform the merge
df = df.merge(inflation_unique[['quarter', 'dk_ann_infl_rate%']], on='quarter', how='left')

In [ ]:
df.head(3)

In [ ]:
inflation_unique

In [ ]:
df.loc[df["quarter"] == '2024Q3']

In [ ]:
df["dk_ann_infl_rate%"].isna().sum()

# 13. Importing inflation rate data file

In [ ]:
morgage = pd.read_excel(r'C:\Users\ insert your path \DK_morgage_rates.xlsx')

In [ ]:
morgage.head(3)

In [ ]:
morgage.dtypes

In [ ]:
morgage["quarter"] = pd.PeriodIndex(morgage["date"], freq="Q")

In [ ]:
morgage.head(3)

In [ ]:
morgage.drop("date", axis=1)

In [ ]:
morgage_unique = morgage.drop_duplicates(subset='quarter')

# Now, perform the merge
df = df.merge(morgage_unique[['quarter', 'yield_on_mortgage_credit_bonds%']], on='quarter', how='left')

In [ ]:
df.head()

In [ ]:
df.loc[df["quarter"] == '2024Q3']

In [ ]:
df.loc[df["quarter"] == '1995Q3']

In [ ]:
df.info()

# 14. Final checks and adjustments

In [ ]:
df["nom_interest_rate%"] = df["nom_interest_rate%"].astype("float32")
df["dk_ann_infl_rate%"] = df["dk_ann_infl_rate%"].astype("float32")
df["yield_on_mortgage_credit_bonds%"] = df["yield_on_mortgage_credit_bonds%"].astype("float32")

In [ ]:
df["area"] = df["area"].astype("category")
df["region"] = df["region"].astype("category")

In [ ]:
df

In [ ]:
df = df.sort_values(by=['date'], ascending=False)

In [ ]:
df

# 15. Exporting

In [ ]:
df.to_csv(r'C:\Users insert your path \DKHousingPrices.csv', index=False)

In [ ]:
df.to_parquet(r'C:\Users\ insert your path \DKHousingPrices.parquet', engine='auto', compression='snappy', index=False)

In [ ]:
df_t = pd.read_parquet(r'C:\Users\Marty\Desktop\DKHousingPrices\Cleaned_files\DKHousingPrices.parquet')

In [ ]:
df_t

In [ ]:
# reducing file size futher but not enough thus point 16...

df_t.to_parquet(r'C:\Users\Marty\Desktop\DKHousingPricesGit\Cleaned_files\DKHousingPrices.parquet2', engine='auto', compression='zstd', index=False)

In [ ]:
df_t.info()

# 16. Extra for spliting parquet in two files

In [3]:
df_t = pd.read_parquet(r'C:\Users\ insert your path \DKHousingPrices.parquet')

In [6]:
df_t.shape

(1507908, 19)

In [10]:

# Define the chunk size (e.g., number of rows per chunk)
chunk_size = 800000  # Adjust as needed based on the size of the dataset

# Define the target directory
target_directory = r"C:\Users\Marty\Desktop\DKHousingPricesGit\Cleaned_files"

# Ensure the target directory exists (create it if not)
os.makedirs(target_directory, exist_ok=True)

# Split the DataFrame into chunks and write each chunk to a new Parquet file
for i in range(0, len(df_t), chunk_size):
    chunk = df_t.iloc[i:i + chunk_size]
    
    # Calculate the current chunk index
    chunk_index = i // chunk_size
    # Construct the full file path for each chunk
    file_name = os.path.join(target_directory, f'file_part_{chunk_index}.parquet')
    
    # Print the name of the file being created to verify unique names
    print(f"Creating: {file_name}")
    
    # Save each chunk as a new Parquet file with zstd compression
    chunk.to_parquet(file_name, compression='zstd')
    
    # Print confirmation
    print(f"Created: {file_name}")

Creating: C:\Users\Marty\Desktop\DKHousingPricesGit\Cleaned_files\file_part_0.parquet
Created: C:\Users\Marty\Desktop\DKHousingPricesGit\Cleaned_files\file_part_0.parquet
Creating: C:\Users\Marty\Desktop\DKHousingPricesGit\Cleaned_files\file_part_1.parquet
Created: C:\Users\Marty\Desktop\DKHousingPricesGit\Cleaned_files\file_part_1.parquet


In [18]:
test = pd.read_parquet(r'C:\Users\Marty\Desktop\DKHousingPricesGit\Cleaned_files\file_part_1.parquet')

In [24]:
test.dtypes

date                                   datetime64[ns]
quarter                                 period[Q-DEC]
house_id                                        int64
house_type                                   category
sales_type                                   category
year_build                                      int16
purchase_price                                  int32
%_change_between_offer_and_purchase           float32
no_rooms                                         int8
sqm                                           float32
sqm_price                                     float32
address                                        object
zip_code                                        int16
city                                           object
area                                         category
region                                       category
nom_interest_rate%                            float32
dk_ann_infl_rate%                             float32
yield_on_mortgage_credit_bon